In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/Project-MONAI/MONAI.git
!cd MONAI/
!pip install -e '.[all]'

Cloning into 'MONAI'...
remote: Enumerating objects: 34371, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 34371 (delta 57), reused 65 (delta 33), pack-reused 34256
Receiving objects: 100% (34371/34371), 64.41 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (27355/27355), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install "monai[einops]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

2023-04-09 19:39:45.532982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 19:39:46.601047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import warnings
warnings.filterwarnings("ignore")
from monai.utils import first, set_determinism

In [ ]:
import os
import shutil
import tempfile
import json
import time
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from tqdm import tqdm

from monai.losses import DiceCELoss
from monai.utils import first
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    Activations,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    RandAffined,
    RandGaussianNoised,
    Invertd
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import UNETR
from monai.apps import download_and_extract
from monai.utils.enums import MetricReduction
from monai import data
from monai.data import decollate_batch
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from functools import partial
from monai.apps import download_and_extract

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


import torch
# import logging
# print_config()
# logging.basicConfig(level=logging.CRITICAL)


In [ ]:
device = torch.device("cuda:0")

Setup data directory

In [ ]:
directory = os.environ.get("MONAI_UNETR_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmproc4ivtc


In [ ]:
# Set deterministic training for reproducibility
set_determinism(seed=0)

Setup transforms for training and validation

In [ ]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-175,
            a_max=250,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
     #augmentation
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[0],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[1],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[2],
            prob=0.10,
        ),
        RandRotate90d(
            keys=["image", "label"],
            prob=0.10,
            max_k=3,
        ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=0.10,
            prob=0.50,
        ),
        RandAffined(
            keys=['image', 'label'], 
            prob=0.5, 
            translate_range=10
        ),
        RandGaussianNoised(
            keys='image', 
            prob=0.5
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["image", "label"], source_key="image"),
    ]
)

Download dataset and format in the folder.

In [ ]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

Task09_Spleen.tar: 1.50GB [01:14, 21.6MB/s]                            

2023-04-09 19:41:11,771 - INFO - Downloaded: /tmp/tmproc4ivtc/Task09_Spleen.tar


2023-04-09 19:41:14,835 - INFO - Verified 'Task09_Spleen.tar', md5: 410d4a301da4e5b2f6f86ec3ddba524e.
2023-04-09 19:41:14,836 - INFO - Writing into directory: /tmp/tmproc4ivtc.


In [ ]:
train_images = sorted(glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_labels = sorted(glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))
data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)]
train_files, val_files = data_dicts[:-9], data_dicts[-9:]


Setup dataloader

In [ ]:
#Define CacheDataset and DataLoader for training and validation
train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=4)
# train_ds = Dataset(data=train_files, transform=train_transforms)

# use batch_size=2 to load images and use RandCropByPosNegLabeld
# to generate 2 x 4 images for network training
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4,pin_memory=True)

val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=4)
# val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1,shuffle=False, num_workers=4, pin_memory=True)


Loading dataset: 100%|██████████| 9/9 [00:08<00:00,  1.10it/s]


## hyperparameter tuning

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import ParameterGrid

training process

In [ ]:
unetr_path = '/content/drive/MyDrive/Winter23/deep_learning/final_project/UNETR'

In [ ]:
def validation(epoch_iterator_val):
    model.eval()
    with torch.no_grad():
        for batch in epoch_iterator_val:
            val_inputs, val_labels = (batch["image"].cuda(), batch["label"].cuda())
            val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            epoch_iterator_val.set_description("Validate (%d / %d Steps)" % (global_step, 10.0))
        mean_dice_val = dice_metric.aggregate().item()
        dice_metric.reset()
    return mean_dice_val


def train(global_step, train_loader, dice_val_best, global_step_best, trial):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["image"].cuda(), batch["label"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description("Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss))
        if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
            epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)
            dice_val = validation(epoch_iterator_val)
            epoch_loss /= step
            epoch_loss_values.append(epoch_loss)
            metric_values.append(dice_val)
            if dice_val > dice_val_best:
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save(model.state_dict(), os.path.join(root_dir, "best_metric_model.pth"))
                unetr_path = '/content/drive/MyDrive/Winter23/deep_learning/final_project/UNETR'
                torch.save(model.state_dict(), os.path.join(unetr_path, f"best_metric_model_{trial}.pth"))
                print(
                    "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val)
                )
            else:
                print(
                    "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(
                        dice_val_best, dice_val
                    )
                )
        global_step += 1
    return global_step, dice_val_best, global_step_best

In [ ]:
hyperparams = {
    'feature_size': [16, 32],
    'mlp_dim': [2024, 3072],
    'num_heads': [12, 16],
    'lr': [0.001, 0.0001],
    'hidden_size': [576,768],
}
hyperparam_grid = list(ParameterGrid(hyperparams))

In [ ]:
len(list(ParameterGrid(hyperparams)))

32

In [ ]:
#Create Model, Loss, Optimizer
device = torch.device("cuda:0")
# create a grid of hyperparameters
hyperparam_grid = list(ParameterGrid(hyperparams))
best_result = 0
best_params ={}
# iterate over the hyperparameter grid
for i, params in enumerate(hyperparam_grid):
    model = UNETR(
        in_channels=1,
        out_channels=2,
        img_size=(96, 96, 96),
        feature_size= params['feature_size'],
        hidden_size= params['hidden_size'],
        mlp_dim= params['mlp_dim'],
        num_heads= params['num_heads'],
        pos_embed="perceptron",
        norm_name="instance",
        res_block=True,
        dropout_rate=0.0,
        ).to(device)

    loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
    torch.backends.cudnn.benchmark = True
    optimizer = torch.optim.AdamW(model.parameters(), lr= params['lr'], weight_decay=1e-5)

    max_iterations = 25000
    eval_num = 500
    post_label = AsDiscrete(to_onehot=14)
    post_pred = AsDiscrete(argmax=True, to_onehot=14)
    dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    global_step = 0
    dice_val_best = 0.0
    global_step_best = 0
    epoch_loss_values = []
    metric_values = []
    while global_step < max_iterations:
        global_step, dice_val_best, global_step_best = train(global_step, train_loader, dice_val_best, global_step_best, i)
    model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
    torch.save(model.state_dict(), os.path.join(unetr_path, f"best_model2_{i+1}.pth"))

    print(f"train completed, best_metric: {dice_val_best:.4f} " f"at iteration: {global_step_best}")
    if dice_val_best> best_result:
        best_result=dice_val_best
        best_params = params

In [ ]:
print(f'best_metric = {best_result}')
print(f'best params: {best_params}')

best_metric = 0.9657 
best params: {'feature_size': 16, 'hidden_size': 768, 'lr': 0.0001, 'mlp_dim': 3072, 'num_heads': 12}}


In [ ]:
# best_metric = 0.9657 
# best params: {'feature_size': 16, 'hidden_size': 768, 'lr': 0.0001, 'mlp_dim': 3072, 'num_heads': 12}}